## 01. 라이브러리 로드

In [1]:
import pandas as pd
import numpy as np

## 02. ifrs 데이터 로드
- ts2000 기반 데이터
    - ifrs_df1 : ifrs_연결 데이터
    - ifrs_df2 : ifrs_별도 데이터

In [2]:
ifrs_df1 = pd.read_excel("data/ts2000/ifrs_연결.xlsx", dtype={'거래소코드': str})
ifrs_df2 = pd.read_excel("data/ts2000/ifrs_별도.xlsx", dtype={'거래소코드': str})

#### ifrs_df1과 ifrs_df2를 결합시키기 위한 컬럼명 재정의

In [3]:
ifrs_df1.rename(columns={'* 발행한 주식총수(*)(IFRS연결)(천원)': '* 발행한 주식총수(*)(IFRS연결)'}, inplace=True)
ifrs_df1.rename(columns={'EBITDA(IFRS)(백만원)': 'EBITDA(IFRS연결)(백만원)'}, inplace=True)
ifrs_df2.rename(columns={'* 발행한 주식총수(*)(IFRS)(주)': '* 발행한 주식총수(*)(IFRS)'}, inplace=True)

#### 회계년도가 12월이 아닌 데이터 삭제

In [4]:
ifrs_df1 = ifrs_df1[ifrs_df1['회계년도'].str.endswith('/12')]
ifrs_df2 = ifrs_df2[ifrs_df2['회계년도'].str.endswith('/12')]

#### ifrs_df1과 ifrs_df2를 결합시키기 위해 접미사 제거 (unify_columns 함수 사용)

In [5]:
key_cols = ['회사명', '거래소코드', '회계년도']

def unify_columns(df, suffix_to_remove):
    df = df.copy()
    df.columns = [col.replace(suffix_to_remove, '') if col not in key_cols else col for col in df.columns]
    return df

ifrs_df1_clean = unify_columns(ifrs_df1, '(IFRS연결)')
ifrs_df2_clean = unify_columns(ifrs_df2, '(IFRS)')

ifrs_df = ifrs_df1_clean.copy()
for col in ifrs_df1_clean.columns:
    if col not in key_cols and col in ifrs_df2_clean.columns:
        ifrs_df[col] = ifrs_df1_clean[col].fillna(ifrs_df2_clean[col])

#### 파생변수 생성
- 자산_평균 : 데이터의 시차를 맞추기 위해 전년도 값과 이번년도 값의 평균값 사용
- 자본_평균 : 데이터의 시차를 맞추기 위해 전년도 값과 이번년도 값의 평균값 사용

In [6]:
ifrs_df['자산_평균'] = ifrs_df.groupby('거래소코드')['자산(*)(천원)'].transform(lambda x: (x + x.shift(1)) / 2)
ifrs_df['자본_평균'] = ifrs_df.groupby('거래소코드')['자본(*)(천원)'].transform(lambda x: (x + x.shift(1)) / 2)

#### 거래소코드와 회계년도를 기준으로 중복 행 삭제 (keep = 'first' 를 사용하여 첫번째 행 살리기)

In [7]:
ifrs_df = ifrs_df.drop_duplicates(subset=['거래소코드', '회계년도'], keep='first')
print(ifrs_df.shape)

(30018, 38)


## 02. gaap 데이터 로드
- ts2000 기반 데이터
    - gaap_df1 : gaap_연결 데이터
    - gaap_df2 : gaap_별도 데이터
- ifrs 연결재무재표를 기준으로 하나 ifrs재무재표에 없는 컬럼들의 값을 위해 gaap데이터 로드

In [8]:
gaap_df1 = pd.read_excel("data/ts2000/gaap_연결.xlsx", dtype={'거래소코드': str})
gaap_df2 = pd.read_excel("data/ts2000/gaap_별도.xlsx", dtype={'거래소코드': str})

In [9]:
gaap_df1 = gaap_df1[gaap_df1['회계년도'].str.endswith('/12')]
gaap_df2 = gaap_df2[gaap_df2['회계년도'].str.endswith('/12')]

In [10]:
key_cols = ['회사명', '거래소코드', '회계년도']

def unify_columns(df, suffix_to_remove):
    df = df.copy()
    df.columns = [col.replace(suffix_to_remove, '') if col not in key_cols else col for col in df.columns]
    return df

gaap_df1_clean = unify_columns(gaap_df1, '(연결)')

gaap_df = gaap_df1_clean.copy()
for col in gaap_df1_clean.columns:
    if col not in key_cols and col in gaap_df2.columns:
        gaap_df[col] = gaap_df1_clean[col].fillna(gaap_df2[col])

In [11]:
gaap_df['자산_평균'] = gaap_df.groupby('거래소코드')['자산(*)(천원)'].transform(lambda x: (x + x.shift(1)) / 2)
gaap_df['자본_평균'] = gaap_df.groupby('거래소코드')['자본(*)(천원)'].transform(lambda x: (x + x.shift(1)) / 2)

In [12]:
gaap_df = gaap_df.drop_duplicates(subset=['거래소코드', '회계년도'], keep='first')
print(gaap_df.shape)

(30018, 38)


#### ifrs_df와 gaap_df를 결합시키기 위해 컬럼명 재정의

In [13]:
ifrs_df.rename(columns={'영업활동으로 인한 현금흐름(간접법)(*)(천원)': '영업활동으로 인한 현금흐름(*)(천원)'}, inplace=True)
ifrs_df.rename(columns={'기말이익잉여금(결손금)(*)(천원)': '이익잉여금(결손금)(*)(천원)'}, inplace=True)
ifrs_df.rename(columns={'* (정상)영업손익(보고서기재)(천원)': '영업이익(손실)(천원)'}, inplace=True)
ifrs_df.rename(columns={'* 발행한 주식총수(*)': '발행주식수(*)(주)'}, inplace=True)
ifrs_df.rename(columns={'EBITDA(IFRS)(백만원)': 'EBITDA(백만원)'}, inplace=True)
gaap_df.rename(columns={'매출액(영업수익)(*)(천원)': '매출액(수익)(*)(천원)'}, inplace=True)
gaap_df.rename(columns={'영업이익증가율': '정상영업이익증가율'}, inplace=True)
gaap_df.rename(columns={'영업이익증가율': '기말이익잉여금(결손금)(*)(천원)'}, inplace=True)
gaap_df.rename(columns={'당기순이익(순손실)(천원)': '당기순이익(손실)(천원)'}, inplace=True)

#### ifrs_df와 gaap_df 결합
- ifrs 위주로  gaap 데이터로 결측치 보완

In [14]:
# 공통 key 컬럼 지정
key_cols = ['회사명', '거래소코드', '회계년도']

# 1. IFRS + GAAP 병합
ts_df = pd.merge(ifrs_df, gaap_df, on=key_cols, how='left', suffixes=('', '_gaap'))

# 2. 결측치 보완: ifrs 값이 결측이면 gaap 값으로 채움
for col in ifrs_df.columns:
    if col in key_cols:
        continue
    gaap_col = f"{col}_gaap"
    if gaap_col in ts_df.columns:
        ts_df[col] = ts_df[col].fillna(ts_df[gaap_col])

# 3. GAAP 보조 컬럼 제거
ts_df = ts_df[ifrs_df.columns]
print(ts_df.shape)

(30018, 38)


In [120]:
ts_df.isna().sum().sort_values(ascending=False)

외국인_주식분포비율               5380
자본_평균                    3097
자산_평균                    3097
EBITDA(백만원)              1132
당좌비율                      986
매출액증가율                    986
정상영업이익증가율                 986
총자본증가율                    986
총자본회전률                    986
자기자본회전률                   986
재고자산회전률                   986
자기자본구성비율                  986
순이익증가율                    986
차입금의존도                    986
총자본순이익률                   986
이자보상배율(이자비용)              986
매출원가 대 매출액비율              986
투자활동으로 인한 현금흐름(*)(천원)     251
이익잉여금(결손금)(*)(천원)         251
발행주식수(*)(주)               251
자산(*)(천원)                 251
자본(*)(천원)                 251
당기순이익(손실)(천원)             251
영업이익(손실)(천원)              251
영업활동으로 인한 현금흐름(*)(천원)     251
부채(*)(천원)                 251
매출액(수익)(*)(천원)            251
유동부채(*)(천원)               251
재고자산(*)(천원)               251
유동자산(*)(천원)               251
비유동자산(*)(천원)              251
재무활동으로 인한 현금흐름(*)(천원)     251
설립일                         0
거래소코드     

## 04. 결측치 처리 및 0 처리
- 외국인_주식분포비율 : 합친 후에도 결측치가 많으므로 제거
- 이자보상배율(이자비용)과 투자활동으로 인한 현금흐름(*)(천원) : 프로젝트에 y값에 해당함으로 결측치이면 분석이 안됨으로 제거
- y값이 2년연속이 필요함으로 회계년도 기준 연속으로 찍히지 않는 거래소코드 리스트 추출 후 제거
- 매출액증가율 : 거래소코드마다 median 값으로 대체
- 자산_평균, 자본_평균 : 결측치 처리 위해 이전값과 이후값으로 대체

In [63]:
ts_df = ts_df.drop(columns=['외국인_주식분포비율'])

In [64]:
ts_df = ts_df.dropna(subset=['이자보상배율(이자비용)', '투자활동으로 인한 현금흐름(*)(천원)'])

In [65]:
# 회계년도 기준 연속으로 찍히지 않는 거래소코드 뽑은 코드
code_tuples = [('418420 ',), ('439090 ',), ('201490 ',), ('314930 ',), ('199730 ',), ('138580 ',), ('107600 ',), 
               ('331920 ',), ('290520 ',), ('088280 ',), ('124500 ',), ('158430 ',), ('179530 ',), ('443670 ',), 
               ('241840 ',), ('227950 ',), ('259630 ',), ('036220 ',), ('011330 ',), ('206650 ',), ('424870 ',), 
               ('160600 ',), ('311320 ',), ('261780 ',), ('402030 ',), ('131100 ',), ('440110 ',), ('208340 ',), 
               ('304840 ',), ('457370 ',), ('086250 ',), ('463480 ',), ('089970 ',), ('024850 ',), ('226590 ',), 
               ('101970 ',), ('396470 ',), ('068100 ',)]

# 공백 포함한 거래소코드 리스트 추출
codes_to_remove = [code[0] for code in code_tuples]

In [66]:
# 위에 코드 기준에서 뽑은 거래소코드 제거
ts_df = ts_df[~ts_df['거래소코드'].isin(codes_to_remove)]
print(ts_df.shape)

(28738, 37)


#### 매출액증가율 0 처리
- 매출액증가율 컬럼이 0이 많고 중요한 컬럼이기에 각 거래소코드마다 median값으로 대체

In [67]:
# 보간 실패 시 median으로 fallback
for 거래소코드 in ts_df['거래소코드'].unique():
    mask = ts_df['거래소코드'] == 거래소코드
    original_na_mask = ts_df.loc[mask, '매출액증가율'].isna()
    
    # 0이 아닌 유효한 값들의 median 계산
    valid_values = ts_df.loc[mask & ~original_na_mask & (ts_df['매출액증가율'] != 0), '매출액증가율']
    
    if len(valid_values) > 0:
        median_val = valid_values.median()
        # 0인 값만 median으로 대체
        ts_df.loc[mask & (ts_df['매출액증가율'] == 0), '매출액증가율'] = median_val

In [68]:
zero_count = (ts_df['매출액증가율'] == 0).sum()
print(f"매출액증가율이 0인 행의 개수: {zero_count}")

매출액증가율이 0인 행의 개수: 81


#### 2013 ~ 2023 까지 분석에 사용함으로 회계년도가 2011/12와 2012/12인 행 제거 

In [122]:
ts_df = ts_df[~ts_df['회계년도'].isin(['2011/12', '2012/12'])]
print(ts_df.shape)

(25913, 38)


#### 자산_평균과 자본_평균 널값 이전값이나 이후값으로 처리

In [70]:
ts_df['자산_평균'] = ts_df.groupby(['회사명', '거래소코드'])['자산_평균'].transform(lambda x: x.bfill().ffill())
ts_df['자본_평균'] = ts_df.groupby(['회사명', '거래소코드'])['자본_평균'].transform(lambda x: x.bfill().ffill())

#### 널값 처리 후에도 자산_평균, 자본_평균 널값인 회사들은 데이터가 1년만 존재하는 기업이기에 제거

In [71]:
# 데이터가 1년만 존재하는 기업이기에 자산_평균, 자본_평균 데이터 드랍
ts_df = ts_df.dropna(subset=['자산_평균'])
ts_df = ts_df.dropna(subset=['자본_평균'])
print(ts_df.shape)

(24782, 37)


In [72]:
ts_df.isna().sum().sort_values(ascending=False)

EBITDA(백만원)              122
회사명                        0
영업이익(손실)(천원)               0
비유동자산(*)(천원)               0
유동자산(*)(천원)                0
재고자산(*)(천원)                0
유동부채(*)(천원)                0
이익잉여금(결손금)(*)(천원)          0
부채(*)(천원)                  0
당기순이익(손실)(천원)              0
순이익증가율                     0
자본(*)(천원)                  0
설립일                        0
발행주식수(*)(주)                0
산업코드                       0
산업명                        0
자산_평균                      0
매출액증가율                     0
정상영업이익증가율                  0
거래소코드                      0
총자본증가율                     0
회계년도                       0
이자보상배율(이자비용)               0
매출원가 대 매출액비율               0
총자본순이익률                    0
영업활동으로 인한 현금흐름(*)(천원)      0
투자활동으로 인한 현금흐름(*)(천원)      0
재무활동으로 인한 현금흐름(*)(천원)      0
자산(*)(천원)                  0
매출액(수익)(*)(천원)             0
차입금의존도                     0
당좌비율                       0
자기자본구성비율                   0
재고자산회전률                    0
자기자본회전률       

## 06. 데이터 결합 - ts 2000데이터가 기준이므로 left 조인
- 시가총액 데이터와 결합

#### 시가총액 데이터 로드

In [73]:
rtn_df = pd.read_excel('data/krx/시가총액.xlsx', dtype={'거래소코드': str})
rtn_df['거래소코드'] = rtn_df['거래소코드'].str.ljust(7)

#### ts2000에 데이터들의 단위가 천원 단위이므로 단위 맞추고 krx에 가져온 데이터이므로 회사명이 달라 제거

In [74]:
rtn_df['시가총액(천원)'] = rtn_df['시가총액'] // 1000
rtn_df = rtn_df.drop(columns=['시가총액'])
rtn_df = rtn_df.drop(columns=['회사명'])
print(rtn_df.shape)

(28457, 3)


In [75]:
merged_df1 = pd.merge(ts_df, rtn_df, on=['거래소코드', '회계년도'], how='left')
print(merged_df1.shape)

(24782, 38)


#### 상장 전인 기업들의 시가총액은 같은 행에 있는 자산(*)(천원)의 값으로 대체

In [76]:
merged_df1['시가총액(천원)'] = merged_df1['시가총액(천원)'].fillna(merged_df1['자산(*)(천원)'])

In [77]:
merged_df1.isna().sum().sort_values(ascending=False)

EBITDA(백만원)              122
회사명                        0
당기순이익(손실)(천원)              0
비유동자산(*)(천원)               0
유동자산(*)(천원)                0
재고자산(*)(천원)                0
유동부채(*)(천원)                0
이익잉여금(결손금)(*)(천원)          0
부채(*)(천원)                  0
영업이익(손실)(천원)               0
자본(*)(천원)                  0
거래소코드                      0
설립일                        0
발행주식수(*)(주)                0
산업코드                       0
산업명                        0
자산_평균                      0
자본_평균                      0
매출액증가율                     0
순이익증가율                     0
정상영업이익증가율                  0
총자본증가율                     0
회계년도                       0
이자보상배율(이자비용)               0
매출원가 대 매출액비율               0
총자본순이익률                    0
영업활동으로 인한 현금흐름(*)(천원)      0
투자활동으로 인한 현금흐름(*)(천원)      0
재무활동으로 인한 현금흐름(*)(천원)      0
자산(*)(천원)                  0
매출액(수익)(*)(천원)             0
차입금의존도                     0
당좌비율                       0
자기자본구성비율                   0
재고자산회전률       

#### 초과수익률 로드

In [78]:
rtn_df2 = pd.read_excel("data/krx/초과수익률.xlsx", dtype={'거래소코드': str})
rtn_df2['거래소코드'] = rtn_df2['거래소코드'].str.ljust(7)

In [79]:
rtn_df2['회계년도'] = rtn_df2['회계년도'].astype(str) + '/12'
print(rtn_df2.shape)

(21252, 5)


In [80]:
merged_df2 = pd.merge(merged_df1, rtn_df2, on=['회사명', '거래소코드', '회계년도'], how='left')

#### 한번 더 중복하는 행 제거

In [81]:
merged_df2 = merged_df2.drop_duplicates(subset=['거래소코드', '회계년도'], keep='first')
print(merged_df2.shape)

(24782, 40)


In [82]:
merged_df2.isna().sum().sort_values(ascending=False)

초과수익률                    3939
수정종가                     3939
EBITDA(백만원)               122
시가총액(천원)                    0
당기순이익(손실)(천원)               0
유동자산(*)(천원)                 0
재고자산(*)(천원)                 0
유동부채(*)(천원)                 0
이익잉여금(결손금)(*)(천원)           0
부채(*)(천원)                   0
영업이익(손실)(천원)                0
자본(*)(천원)                   0
자본_평균                       0
거래소코드                       0
설립일                         0
발행주식수(*)(주)                 0
산업코드                        0
산업명                         0
자산_평균                       0
비유동자산(*)(천원)                0
회사명                         0
순이익증가율                      0
자산(*)(천원)                   0
회계년도                        0
이자보상배율(이자비용)                0
매출원가 대 매출액비율                0
총자본순이익률                     0
영업활동으로 인한 현금흐름(*)(천원)       0
투자활동으로 인한 현금흐름(*)(천원)       0
재무활동으로 인한 현금흐름(*)(천원)       0
매출액(수익)(*)(천원)              0
정상영업이익증가율                   0
차입금의존도                      0
당좌비율      

#### 수정종가 널값 제거

In [83]:
merged_df2 = merged_df2.dropna(subset=['수정종가'])
print(merged_df2.shape)

(20843, 40)


In [84]:
merged_df2.isna().sum().sort_values(ascending=False)

EBITDA(백만원)              114
거래소코드                      0
유동자산(*)(천원)                0
재고자산(*)(천원)                0
유동부채(*)(천원)                0
이익잉여금(결손금)(*)(천원)          0
부채(*)(천원)                  0
영업이익(손실)(천원)               0
당기순이익(손실)(천원)              0
자본(*)(천원)                  0
회사명                        0
설립일                        0
발행주식수(*)(주)                0
산업코드                       0
산업명                        0
자산_평균                      0
자본_평균                      0
시가총액(천원)                   0
수정종가                       0
비유동자산(*)(천원)               0
매출액증가율                     0
순이익증가율                     0
자산(*)(천원)                  0
회계년도                       0
이자보상배율(이자비용)               0
매출원가 대 매출액비율               0
총자본순이익률                    0
영업활동으로 인한 현금흐름(*)(천원)      0
투자활동으로 인한 현금흐름(*)(천원)      0
재무활동으로 인한 현금흐름(*)(천원)      0
매출액(수익)(*)(천원)             0
정상영업이익증가율                  0
차입금의존도                     0
당좌비율                       0
자기자본구성비율      

#### 종목별_연간변동성 데이터 로드
- 데이터 결합 위해 컬럼명과 값 재정의

In [85]:
volatility_df = pd.read_excel("data/krx/연간변동성.xlsx", dtype={'티커': str})
volatility_df['거래소코드'] = volatility_df['티커'].str.ljust(7)
volatility_df['회계년도'] = volatility_df['회계년도'].astype(str) + '/12'
volatility_df = volatility_df.drop(columns=['티커'])
print(volatility_df.shape)

(31236, 3)


In [86]:
dataset1 = pd.merge(merged_df2, volatility_df, on=['거래소코드', '회계년도'], how='left')
print(dataset1.shape)

(20843, 41)


In [87]:
dataset1.isna().sum().sort_values(ascending=False)

EBITDA(백만원)              114
회사명                        0
비유동자산(*)(천원)               0
재고자산(*)(천원)                0
유동부채(*)(천원)                0
이익잉여금(결손금)(*)(천원)          0
부채(*)(천원)                  0
영업이익(손실)(천원)               0
당기순이익(손실)(천원)              0
자본(*)(천원)                  0
설립일                        0
발행주식수(*)(주)                0
산업코드                       0
산업명                        0
자산_평균                      0
자본_평균                      0
시가총액(천원)                   0
수정종가                       0
초과수익률                      0
유동자산(*)(천원)                0
매출액증가율                     0
거래소코드                      0
순이익증가율                     0
회계년도                       0
이자보상배율(이자비용)               0
매출원가 대 매출액비율               0
총자본순이익률                    0
영업활동으로 인한 현금흐름(*)(천원)      0
투자활동으로 인한 현금흐름(*)(천원)      0
재무활동으로 인한 현금흐름(*)(천원)      0
자산(*)(천원)                  0
매출액(수익)(*)(천원)             0
차입금의존도                     0
당좌비율                       0
자기자본구성비율      

## 07. 파생변수 생성
- ebitda(천원) - 단위맞추기
    - ebitda(백만원) 제거
- 설립년도
- 기업업력
    - 설립일과 설립년도 컬럼 제거
- 알트만 k2변수
    - x1, x2, x3, x4
- roe, roa, pcr, psr
    - ts2000에 없거나 최고, 최저로 나와서 생성
- ln(자산), ln(매출액), ln(시가총액), ln(기업업력)
    - 변수의 스케일 축소 및 정규화
- 잉여현금흐름 비율

In [88]:
dataset1['EBITDA(천원)'] = dataset1['EBITDA(백만원)'] * 1000
dataset1['설립년도'] = dataset1['설립일'].str[:4].astype(int)
dataset1['기업업력'] = dataset1['회계년도'].str[:4].astype(int) - dataset1['설립년도']
dataset1 = dataset1.drop(columns=['설립일', '설립년도'])
dataset1 = dataset1.drop(columns='EBITDA(백만원)')

In [89]:
dataset1['x1'] = np.log(dataset1['자산(*)(천원)'] + 1e-10)
dataset1['x2'] = np.log((dataset1['매출액(수익)(*)(천원)'] / dataset1['자산_평균']) + 1e-10)
dataset1['x3'] = dataset1['이익잉여금(결손금)(*)(천원)'] / dataset1['자산_평균']
dataset1['x4'] = dataset1['시가총액(천원)'] / dataset1['부채(*)(천원)']

In [90]:
dataset1['roe'] = (dataset1['당기순이익(손실)(천원)'] / dataset1['자본_평균']) * 100   
dataset1['roa'] = (dataset1['당기순이익(손실)(천원)'] / dataset1['자산_평균']) * 100
dataset1['pcr'] = dataset1['수정종가'] / (dataset1['영업활동으로 인한 현금흐름(*)(천원)'] / dataset1['수정종가'])
dataset1['psr'] = dataset1['수정종가'] / (dataset1['매출액(수익)(*)(천원)'] / dataset1['발행주식수(*)(주)'])

In [91]:
dataset1['ln(자산)'] = np.log(dataset1['자산(*)(천원)'] + 1e-10)
dataset1['ln(매출액)'] = np.log(dataset1['매출액(수익)(*)(천원)'] + 1e-10)
dataset1['ln(시가총액)'] = np.log(dataset1['시가총액(천원)'] + 1e-10)
dataset1['ln(기업업력)'] = np.log(dataset1['기업업력'] + 1e-10)
dataset1['잉여현금흐름 비율'] = dataset1['영업활동으로 인한 현금흐름(*)(천원)'] / dataset1['자산_평균']

In [97]:
print(dataset1.shape)
dataset1.isna().sum().sort_values(ascending=False)

(20843, 54)


EBITDA(천원)               114
회사명                        0
기업업력                       0
자본(*)(천원)                  0
발행주식수(*)(주)                0
산업코드                       0
산업명                        0
자산_평균                      0
자본_평균                      0
시가총액(천원)                   0
수정종가                       0
초과수익률                      0
연간변동성                      0
x1                         0
거래소코드                      0
x2                         0
x3                         0
x4                         0
roe                        0
roa                        0
pcr                        0
psr                        0
ln(자산)                     0
ln(매출액)                    0
ln(시가총액)                   0
ln(기업업력)                   0
당기순이익(손실)(천원)              0
영업이익(손실)(천원)               0
부채(*)(천원)                  0
이익잉여금(결손금)(*)(천원)          0
회계년도                       0
이자보상배율(이자비용)               0
매출원가 대 매출액비율               0
총자본순이익률                    0
영업활동으로 인한 현금흐름

## 08. 데이터 저장

In [ ]:
dataset1.to_excel("data/원본데이터셋.xlsx", index=False)